In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [35]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self, rootpath=False):
        subclasses = self.get_subclasses(rootpath=False)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples(rootpath))
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        return self.parse_triples(union_flag, subclass_of, rootpath)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
    
    def extract_ns(self):
        '''
        Extracts namespace of an ontology
        '''
        ns = self.ontology_obj.getElementsByTagName("rdf:RDF")[0].getAttribute("xmlns")
        if ns[-1] == "#":
            return ns
        return self.ontology_obj.doctype.entities.item(0).firstChild.nodeValue
    
    def get_subclasses(self, rootpath=False):
        subclasses = [(self.extract_ID(a, not rootpath), self.extract_ID(b, not rootpath), c, d) for (a,b,c,d) in self.subclasses]
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [36]:
alignment_folder = "german_datasets/mapping freizeit/"

ontologies_in_alignment = []

# Load reference alignments 
def load_alignments(folder):
    gt = []
    path = [folder + l for l in os.listdir(folder) if l.endswith(".txt")][0]
    mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
    for mapping in mappings:
        src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
        targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
        ontologies_in_alignment.append((folder + src, folder + targ))
        src = src.rsplit(".",1)[0].replace(".", "_").lower()
        targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
        lines = [["_".join(row.strip().split(":")[0].replace(",-", "_").split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                 for line in mapping.split("\n") if line.startswith(" -")]
        lines = [[src + "#" + line[0], targ + "#" + line[1]] for line in lines]
        gt.append(lines)
    return gt

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)


In [49]:
reference_alignments

[[['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Artistik',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Artistik'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Kerzen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Kerzen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Malen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Malen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Papier',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Papier'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Perlen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Perlen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Serviettentechnik',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Serviettentechnik'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Stempeltechnik',
   'google_frei

In [18]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology(l[0])
    ont2 = Ontology(l[1])
    
    ent1 = ont1.get_classes()
    ent2 = ont2.get_classes()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    pre1 = l[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    pre2 = l[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    print (pre1, pre2)
    all_mappings.extend([(pre1 + "#" + el[0], pre2 + "#" + el[1]) for el in mappings])
    

data = {mapping: False for mapping in all_mappings}
reference_alignments = [tuple(alignment) for alignment in reference_alignments]
s = set(all_mappings)
for mapping in set(reference_alignments):
    if mapping in s:
        data[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data[mapping] = True
        else:
            print (mapping)

dmoz_freizeit google_freizeit


In [48]:
def write_results(final_list):
    ont_name_parsed1 = Ontology(ont_name1).extract_ns()
    ont_name_parsed2 = Ontology(ont_name2).extract_ns()
    ont_name1_pre = ont_name1 if (ont_name1.startswith("http://") or ont_name1.startswith("https://")) else "file://" + ont_name1
    ont_name2_pre = ont_name2 if (ont_name2.startswith("http://") or ont_name2.startswith("https://")) else "file://" + ont_name2
    rdf = \
    """<?xml version='1.0' encoding='utf-8' standalone='no'?>
<rdf:RDF xmlns='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'
         xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'
         xmlns:xsd='http://www.w3.org/2001/XMLSchema#'
         xmlns:align='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'>
<Alignment>
  <xml>yes</xml>
  <level>0</level>
  <type>**</type>
  <onto1>
    <Ontology rdf:about="{}">
      <location>{}</location>
    </Ontology>
  </onto1>
  <onto2>
    <Ontology rdf:about="{}">
      <location>{}</location>
    </Ontology>
  </onto2>""".format(ont_name_parsed1.split("#")[0], ont_name1_pre, ont_name_parsed2.split("#")[0], ont_name2_pre)
    for (a,b,score) in final_list:
        mapping = """
  <map>
    <Cell>
      <entity1 rdf:resource='{}'/>
      <entity2 rdf:resource='{}'/>
      <relation>=</relation>
      <measure rdf:datatype='http://www.w3.org/2001/XMLSchema#float'>{}</measure>
    </Cell>
  </map>""".format(ont_name_parsed1 + "#".join(a.split("#")[1:]), ont_name_parsed2 + "#".join(b.split("#")[1:]), score)
        rdf += mapping
    rdf += """
</Alignment>
</rdf:RDF>"""
    return rdf


# Load reference alignments 
def load_alignments(folder):
    gt = []
    path = [folder + l for l in os.listdir(folder) if l.endswith(".txt")][0]
    mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
    for mapping in mappings:
        src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
        targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
        ontologies_in_alignment.append((folder + src, folder + targ))
        src = src.rsplit(".",1)[0].replace(".", "_").lower()
        targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
        lines = [["_".join(row.strip().split(":")[0].replace(",-", "_").split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                 for line in mapping.split("\n") if line.startswith(" -")]
        lines = [[src + "#" + line[0], targ + "#" + line[1], 1.0] for line in lines]
        gt.append(lines)
    return gt

alignment_folder = "german_datasets/mapping webdirectory/"
ontologies_in_alignment = []

final_list = load_alignments(alignment_folder)

ontologies_in_alignment = [('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl')]

for i,ont_pair in enumerate(ontologies_in_alignment):
    ont_name1, ont_name2 = ont_pair[0], ont_pair[1]
    rdf = write_results(final_list[i])
    c = "_".join(ont_name1.split("/")[-1].split(".")[:-1]) + "-" + "_".join(ont_name2.split("/")[-1].split(".")[:-1]) + ".rdf"
    f = open("/data/Vivek/IBM/VeeAlign/datasets/web-directory/alignments/" + c, "w+")
    f.write(rdf)
    f.close()
    
    
    

In [2]:


def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_") if el])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_") if el])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings([parse(el) for el in keys])))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


NameError: name 'all_mappings' is not defined

In [4]:

filtered_dict = {}
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

extracted_elems = []
mapping_ont = {}
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology(ont_name)
    entities = ont.get_classes()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    ont_name = ont_name.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    mapping_ont[ont_name] = ont
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = []
for word in extracted_elems:
    ont_name = word.split("#")[0]
    elem = word.split("#")[1]
    inp.append(parse(mapping_ont[ont_name].mapping_dict.get(elem, elem)))

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

embeds = np.array(extractUSEEmbeddings(inp))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


Total number of extracted unique classes and properties from entire RA set:  3032


In [65]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl


In [69]:
max_paths

1

In [68]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append(e2)
            neighbours_dict[e2][2].append(e1)
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append(e2)
            neighbours_dict[e2][3].append(e1)
        elif d == "Subclass":
            neighbours_dict[e2][1].append(e1)
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    for entity in neighbours_dict:
        neighbours_dict[entity][1] = [neighbours_dict[entity][1]]
        neighbours_dict[entity][2] = [neighbours_dict[entity][2]]
        neighbours_dict[entity][3] = [neighbours_dict[entity][3]]
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit_bagofnbrs.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl


In [77]:
leb_data = pickle.load(open("Input/data_freizeit.pkl", "rb"))[0]
data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts = pickle.load(open("Input/data_webdir.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("Input/data_fre_webdir_bagofnbrs.pkl", "wb"))

In [19]:
leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, _ = pickle.load(open("../Input/data_webdir_leb.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("../Input/data_webdir_leb_bagofnbrs.pkl", "wb"))
# pickle.dump([pickle.load(open("Input/data_freizeit.pkl", "rb"))[0],
#              pickle.load(open("../Input/data_webdir.pkl", "rb"))[0],
#              emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)

In [59]:
neighbours_dicts

{'yahoo_small#Firmen_Sport_Motorsport': [[['yahoo_small#Firmen_Sport_Motorsport',
    'yahoo_small#Firmen_Sport',
    'yahoo_small#Firmen']],
  [['yahoo_small#Firmen_Sport_Motorsport_Fanartikel'],
   ['yahoo_small#Firmen_Sport_Motorsport_Kartsport']],
  [],
  []],
 'yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit_Bildung-und-Ausbildung': [[['yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit_Bildung-und-Ausbildung',
    'yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit',
    'yahoo_small#Firmen_Gesundheit_Alternativmedizin',
    'yahoo_small#Firmen_Gesundheit',
    'yahoo_small#Firmen']],
  [],
  [],
  []],
 'yahoo_small#Firmen_Sport_Radsport_Mountain-Biking': [[['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking',
    'yahoo_small#Firmen_Sport_Radsport',
    'yahoo_small#Firmen_Sport',
    'yahoo_small#Firmen']],
  [['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking_Komponenten'],
   ['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking_Hersteller']],
  [],

In [62]:
np.max([len([nbr_type for nbr_type in elem if flatten(nbr_type)]) for elem in neighbours_dicts.values()])

2

In [27]:
# max_paths
sorted(Counter(flatten([[len(nbr_type) for nbr_type in elem] for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())


[(0, 8658),
 (1, 3486),
 (2, 148),
 (3, 99),
 (4, 69),
 (5, 33),
 (6, 32),
 (7, 37),
 (8, 12),
 (9, 11),
 (10, 6),
 (11, 6),
 (12, 8),
 (13, 7),
 (15, 4),
 (16, 3),
 (17, 3),
 (18, 1),
 (19, 2),
 (20, 4),
 (21, 5),
 (22, 3),
 (23, 1),
 (24, 2),
 (26, 3),
 (38, 1),
 (41, 1),
 (42, 1),
 (48, 1),
 (49, 1)]

In [28]:
sorted(Counter(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())

[(1, 3156),
 (2, 25),
 (3, 426),
 (4, 1243),
 (5, 1054),
 (6, 329),
 (7, 53),
 (8, 26)]

In [ ]:
embedding_dim = np.array(emb_vals).shape[1]
        
name_embedding = nn.Embedding(len(emb_vals), embedding_dim)
name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
name_embedding.weight.requires_grad = False

cosine_sim_layer = nn.CosineSimilarity(dim=1)
output = nn.Linear(2*embedding_dim, 300)
max_pathlen = np.array(list(neighbours_dicts_pathpadded.values())).shape[3]
v = nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)]))

nodes = torch.randint(0, len(emb_vals), size=(10,2))
features= torch.randint(0, len(emb_vals), size=(10,2,4,22,6))

nodes = nodes.permute(1,0) # 2 * batch_size
features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
for i in range(2):
    node_emb = name_embedding(nodes[i]) # batch_size * 512
    feature_emb = name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512

    path_weights = torch.sum(torch.sum(node_emb[:, None, None, None, :] * feature_emb, dim=-1), dim=-1)
    best_path_indices = torch.max(path_weights, dim=2)[1][(..., ) + (None, ) * 3]
    best_path_indices = best_path_indices.expand(-1, -1, -1, max_pathlen,  embedding_dim)
    best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
    # Another way: 
    # path_weights = masked_softmax(path_weights)
    # best_path = torch.sum(path_weights.unsqueeze(-1) * feature_emb, dim=2)

    node_weights = torch.sum(node_emb[:, None, None, :] * best_path, dim=-1).unsqueeze(-1)
    torch.matmul(v, node_weights * best_path)

In [ ]:
def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))

attended_path = torch.bmm(a.reshape(-1, 1, 512), b.reshape(-1, 512, 1))
attended_path = attended_path.reshape(-1, 4, 22, 6)
path_weights = masked_softmax(torch.sum(attended_path, dim=-1))
path_weights.shape
# best_path = torch.sum(path_weights[:, :, :, None, None] * a, dim=2)

In [ ]:
indices = d[(..., ) + (None, ) * 3].expand(-1, -1, -1, 6, 512)
e = torch.gather(b, 2, indices).squeeze(2)

f = torch.sum(a[:,None,None,:] *e,dim=-1).unsqueeze(-1)

g = (f*e)

h = torch.sum((v[None,None,:,None] * g), dim=2)

In [ ]:
c = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,1,6,512))
d = b * c


In [ ]:
e = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)


In [ ]:
fo

In [24]:
len(data)

3318178

In [ ]:
ctimes, dtimes = [], []
for i in range(100):
    a = torch.randn((10,4,22,6,512))
    b = torch.randn((10,4,1,6,512))
    t = time.time()
    c = a * b
    ctimes.append(time.time()-t)
    t = time.time()
    d = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)
    dtimes.append(time.time()-t)

In [ ]:
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))
c = a * torch.sum(b,dim=2).unsqueeze(2)

In [23]:
arr = [('german_datasets_copy/lebensmittel/Google.Lebensmittel.owl',
  'german_datasets_copy/lebensmittel/web.Lebensmittel.owl')]
for ont_pair in arr:
    a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    !rm -rf $c
    os.mkdir(c)
    java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a) + \
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
    process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
pred_logmap = [[el.split("/")[-1] for el in l.split("\t")[:-1]] for l in open(os.path.abspath(c + "/") + "/logmap2_mappings.tsv",  "r").read().split("\n")[:-1] if not l.startswith("Optional")]


KeyboardInterrupt: 

In [ ]:
gt_mappings = []
for elem in pred_logmap:
    gt_mappings.append(tuple([el.split("#")[0].replace(".v2", "").rsplit(".",1)[0].replace(".", "_").lower() + "#" + el.split("#")[1] for el in elem]))


In [ ]:
data_orig = pickle.load(open("../Input/data_lebensmittel.pkl", "rb"))[0]
data_logmap = {}
for key in data_orig:
    data_logmap[key] = False
s = set(list(data_logmap.keys()))
gt_mappings = [tuple(pair) for pair in gt_mappings]
for mapping in gt_mappings:
    
    if mapping in s:
        data_logmap[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data_logmap[mapping] = True
        else:
            print (mapping)

In [ ]:
all_metrics = []
def return_test_data(data, i):
    data_t = {elem: data[elem] for elem in data if data[elem]}
    data_f = {elem: data[elem] for elem in data if not data[elem]}

    data_t_items = list(data_t.keys())
    data_f_items = list(data_f.keys())

    test_data_t = data_t_items[int((0.2*i)*len(data_t)):int((0.2*i + 0.2)*len(data_t))]
    test_data_f = data_f_items[int((0.2*i)*len(data_f)):int((0.2*i + 0.2)*len(data_f))]
    
    test_data = {}
    for elem in test_data_t:
        test_data[elem] = True
    for elem in test_data_f:
        test_data[elem] = False
    return test_data

for i in range(5):
    test_gt = return_test_data(data_orig, i)
    test_logmap = {elem: data_logmap[elem] for elem in test_gt}
    tp = len([elem for elem in test_gt if test_gt[elem] and test_logmap[elem]])
    fp = len([elem for elem in test_logmap if not test_gt[elem] and test_logmap[elem]])
    fn = len([elem for elem in test_logmap if test_gt[elem] and not test_logmap[elem]])
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        continue
    all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    

In [ ]:
np.mean(all_metrics, axis=0)

In [ ]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []

for i in list(range(0, ontologies_in_alignment, 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        print (ont_pair)
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
        java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a)
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

In [ ]:
ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment][:-1] + [["human", "mouse"]]

In [77]:
output = """Output_att12_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.77256318 0.70163934 0.73539519 0.71476286 0.75725407]
Output_att12_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72786885 0.72786885 0.72786885 0.72786885 0.72786885]
Output_att12_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.7284345  0.74754098 0.73786408 0.74363992 0.73217726]
Output_att12_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.7147541 0.7147541 0.7147541 0.7147541 0.7147541]
Output_att12_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74257426 0.73770492 0.74013158 0.73867367 0.74159525]
Output_att12_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74817518 0.67213115 0.70811744 0.68607764 0.73162027]
Output_att12_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.69578313 0.75737705 0.72527473 0.74420103 0.7072872 ]
Output_att12_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72727273 0.73442623 0.73083197 0.73298429 0.72869226]
Output_att12_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74576271 0.72131148 0.73333333 0.72607261 0.74074074]
Output_att12_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71717172 0.69836066 0.7076412  0.70204351 0.71332887]
Output_att12_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76383764 0.67868852 0.71875    0.69416499 0.74514039]
Output_att12_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75457875 0.67540984 0.71280277 0.68988614 0.7372942 ]
Output_att12_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76027397 0.72786885 0.74371859 0.73412698 0.75356415]
Output_att12_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72131148 0.72131148 0.72131148 0.72131148 0.72131148]
Output_att12_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75254237 0.72786885 0.74       0.73267327 0.74747475]
Output_att12_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.75179856 0.6852459  0.71698113 0.6975968  0.73747354]
Output_att12_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76190476 0.68196721 0.71972318 0.69658406 0.7444524 ]
Output_att12_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75172414 0.7147541  0.73277311 0.7218543  0.7440273 ]
Output_att12_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74       0.72786885 0.7338843  0.73026316 0.73754153]
Output_att12_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.7254902  0.72786885 0.72667758 0.72739187 0.72596468]
Output_att12_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.71656051 0.73770492 0.726979   0.73337679 0.72069186]
Output_att12_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72459016 0.72459016 0.72459016 0.72459016 0.72459016]
Output_att12_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76156584 0.70163934 0.73037543 0.71285809 0.74877537]
Output_att12_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75272727 0.67868852 0.7137931  0.69230769 0.7366548 ]
Output_att12_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.77003484 0.72459016 0.74662162 0.73324486 0.76049553]
Output_att12_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72786885 0.72786885 0.72786885 0.72786885 0.72786885]
Output_att12_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.71698113 0.74754098 0.73194222 0.74122237 0.72289157]
Output_att12_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71518987 0.74098361 0.72785829 0.73567708 0.72020395]
Output_att12_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74740484 0.70819672 0.72727273 0.71570577 0.73921971]
Output_att12_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.73026316 0.72786885 0.72906404 0.72834646 0.72978304]
Output_att13_5_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.74825175 0.70163934 0.72419628 0.71049137 0.7384403 ]
Output_att13_5_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.75167785 0.73442623 0.74295191 0.73781291 0.74816299]
Output_att13_5_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.74013158 0.73770492 0.73891626 0.73818898 0.73964497]
Output_att13_5_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.77902622 0.68196721 0.72727273 0.69939475 0.7574654 ]
Output_att13_5_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.71518987 0.74098361 0.72785829 0.73567708 0.72020395]
Output_att13_5_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.73311897 0.74754098 0.74025974 0.74461137 0.73595868]
Output_att13_5_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.72413793 0.75737705 0.74038462 0.75048733 0.73055028]
Output_att13_5_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.74750831 0.73770492 0.74257426 0.73964497 0.74552684]
Output_att13_5_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.73311897 0.74754098 0.74025974 0.74461137 0.73595868]
Output_att13_5_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.78700361 0.7147541  0.74914089 0.72812291 0.77140835]
Output_att13_5_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.75510204 0.72786885 0.74123539 0.7331572  0.74949359]
Output_att13_8_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.76573427 0.71803279 0.74111675 0.72709163 0.75569358]
Output_att13_8_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.73754153 0.72786885 0.73267327 0.72978304 0.73558648]
Output_att13_8_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77083333 0.72786885 0.74873524 0.73607427 0.7618394 ]
Output_att13_8_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.74750831 0.73770492 0.74257426 0.73964497 0.74552684]
Output_att13_8_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.72727273 0.73442623 0.73083197 0.73298429 0.72869226]
Output_att13_8_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.71786834 0.75081967 0.73397436 0.7439896  0.72422517]
Output_att13_8_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.73856209 0.74098361 0.73977087 0.74049803 0.73904513]
Output_att13_8_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.76923077 0.72131148 0.74450085 0.73041169 0.75914424]
Output_att13_8_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.74647887 0.69508197 0.71986418 0.70478723 0.73560028]
Output_att13_8_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.78136201 0.7147541  0.74657534 0.72715143 0.76706545]
Output_att13_8_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.7284345  0.74754098 0.73786408 0.74363992 0.73217726]
Output_att18_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.70121951 0.75409836 0.72669826 0.74289406 0.71119357]
Output_att18_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.70426829 0.75737705 0.72985782 0.74612403 0.71428571]
Output_att18_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74333333 0.73114754 0.73719008 0.73355263 0.74086379]
Output_att18_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72333333 0.71147541 0.71735537 0.71381579 0.72093023]
Output_att18_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75912409 0.68196721 0.71848014 0.6961178  0.74232691]
Output_att18_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.73591549 0.6852459  0.70967742 0.69481383 0.72519084]
Output_att18_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74394464 0.70491803 0.72390572 0.71239231 0.7357974 ]
Output_att18_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.71518987 0.74098361 0.72785829 0.73567708 0.72020395]
Output_att18_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.73376623 0.74098361 0.73735726 0.7395288  0.73519844]
Output_att18_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.68768769 0.75081967 0.71786834 0.73728268 0.69945021]
Output_att18_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76895307 0.69836066 0.73195876 0.71142285 0.7537155 ]
Output_att18_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74394464 0.70491803 0.72390572 0.71239231 0.7357974 ]
Output_att18_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.73666667 0.72459016 0.73057851 0.72697368 0.73421927]
Output_att18_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72786885 0.72786885 0.72786885 0.72786885 0.72786885]
Output_att18_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75337838 0.73114754 0.74209651 0.73548813 0.74882471]
Output_att18_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70253165 0.72786885 0.71497585 0.72265625 0.70745698]
Output_att18_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75432526 0.7147541  0.73400673 0.72233267 0.74606434]
Output_att18_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74827586 0.71147541 0.72941176 0.71854305 0.74061433]
Output_att18_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.775      0.71147541 0.74188034 0.72333333 0.76140351]
Output_att18_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72012579 0.75081967 0.73515249 0.74447334 0.72606214]
Output_att18_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.7125     0.74754098 0.7296     0.74025974 0.7192429 ]
Output_att18_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72964169 0.73442623 0.73202614 0.73346431 0.73059361]
Output_att18_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.77153558 0.67540984 0.72027972 0.6926698  0.75018208]
Output_att18_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.73702422 0.69836066 0.71717172 0.70576541 0.72895277]
Output_att18_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72012579 0.75081967 0.73515249 0.74447334 0.72606214]
Output_att18_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.74652778 0.70491803 0.72512648 0.71286472 0.73781743]
Output_att18_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75084175 0.73114754 0.74086379 0.7350033  0.74681849]
Output_att18_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69620253 0.72131148 0.70853462 0.71614583 0.70108349]
Output_att18_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.71153846 0.72786885 0.71961102 0.72454308 0.71474565]
Output_att18_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75824176 0.67868852 0.71626298 0.6932351  0.7408733 ]
Output_att21_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76124567 0.72131148 0.74074074 0.72895958 0.75290897]
Output_att21_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.6996997  0.76393443 0.73040752 0.75016098 0.71166768]
Output_att21_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74       0.72786885 0.7338843  0.73026316 0.73754153]
Output_att21_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69135802 0.73442623 0.71224165 0.7253886  0.69956277]
Output_att21_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76369863 0.73114754 0.74706868 0.73743386 0.75695859]
Output_att21_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77186312 0.66557377 0.71478873 0.68442347 0.74797347]
Output_att21_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.70090634 0.76065574 0.72955975 0.74790458 0.71209331]
Output_att21_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.73       0.71803279 0.72396694 0.72039474 0.72757475]
Output_att21_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75342466 0.72131148 0.73701843 0.72751323 0.74677529]
Output_att21_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71019108 0.73114754 0.72051696 0.72685789 0.71428571]
Output_att21_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75601375 0.72131148 0.73825503 0.72799471 0.74880871]
Output_att21_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.72       0.70819672 0.71404959 0.71052632 0.71760797]
Output_att21_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76642336 0.68852459 0.7253886  0.70281124 0.74946467]
Output_att21_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.7184466  0.72786885 0.72312704 0.72596468 0.72031149]
Output_att21_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75172414 0.7147541  0.73277311 0.7218543  0.7440273 ]
Output_att21_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69846154 0.7442623  0.72063492 0.73462783 0.70716511]
Output_att21_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75539568 0.68852459 0.72041166 0.70093458 0.74100212]
Output_att21_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74204947 0.68852459 0.71428571 0.69860279 0.73068894]
Output_att21_5_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.73883162 0.70491803 0.72147651 0.71144937 0.73179033]
Output_att21_5_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.74671053 0.7442623  0.7454844  0.74475066 0.74621959]
Output_att21_5_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77003484 0.72459016 0.74662162 0.73324486 0.76049553]
Output_att21_5_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.75778547 0.71803279 0.73737374 0.72564612 0.74948665]
Output_att21_5_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.74257426 0.73770492 0.74013158 0.73867367 0.74159525]
Output_att21_5_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.71019108 0.73114754 0.72051696 0.72685789 0.71428571]
Output_att21_5_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.76450512 0.73442623 0.74916388 0.74025116 0.75829384]
Output_att21_5_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.75945017 0.72459016 0.74161074 0.73130377 0.75221239]
Output_att21_5_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.72277228 0.71803279 0.72039474 0.71897571 0.72181938]
Output_att21_5_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.77003484 0.72459016 0.74662162 0.73324486 0.76049553]
Output_att21_5_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.7266881  0.74098361 0.73376623 0.73807969 0.72950291]
Output_att21_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72516556 0.71803279 0.72158155 0.71944809 0.72372769]
Output_att21_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.74394464 0.70491803 0.72390572 0.71239231 0.7357974 ]
Output_att21_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74013158 0.73770492 0.73891626 0.73818898 0.73964497]
Output_att21_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72875817 0.73114754 0.7299509  0.73066841 0.72923479]
Output_att21_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.77067669 0.67213115 0.71803853 0.6897712  0.74872169]
Output_att21_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.71428571 0.72131148 0.7177814  0.71989529 0.7156799 ]
Output_att21_8_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.78358209 0.68852459 0.73298429 0.70564516 0.76252723]
Output_att21_8_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.74666667 0.73442623 0.74049587 0.73684211 0.74418605]
Output_att21_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.69604863 0.75081967 0.72239748 0.73918657 0.7063541 ]
Output_att21_8_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77224199 0.71147541 0.74061433 0.72285143 0.75927222]
Output_att21_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.75886525 0.70163934 0.72913118 0.71238349 0.74668528]
Output_att21_8_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.77536232 0.70163934 0.73666093 0.71524064 0.75940383]
Output_att21_8_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.74324324 0.72131148 0.73211314 0.72559367 0.73875084]
Output_att21_8_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.73089701 0.72131148 0.72607261 0.72320842 0.72895958]
Output_att21_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.71293375 0.74098361 0.7266881  0.73519844 0.71837254]
Output_att21_8_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.77210884 0.7442623  0.75792988 0.74966975 0.76637407]
Output_att21_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70440252 0.73442623 0.71910112 0.72821847 0.71020926]
Output_att21_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76383764 0.67868852 0.71875    0.69416499 0.74514039]
Output_att21_8_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.78832117 0.70819672 0.74611399 0.72289157 0.77087794]
Output_att21_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75912409 0.68196721 0.71848014 0.6961178  0.74232691]
Output_att21_8_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.72435897 0.74098361 0.73257699 0.73759791 0.72762395]
Output_att21_8_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.78277154 0.6852459  0.73076923 0.70275723 0.76110706]
Output_att21_8_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.72523962 0.7442623  0.73462783 0.74037834 0.72896596]
Output_att24_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76124567 0.72131148 0.74074074 0.72895958 0.75290897]
Output_att24_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.6996997  0.76393443 0.73040752 0.75016098 0.71166768]
Output_att24_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75337838 0.73114754 0.74209651 0.73548813 0.74882471]
Output_att24_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69135802 0.73442623 0.71224165 0.7253886  0.69956277]
Output_att24_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76369863 0.73114754 0.74706868 0.73743386 0.75695859]
Output_att24_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77186312 0.66557377 0.71478873 0.68442347 0.74797347]
Output_att24_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.70090634 0.76065574 0.72955975 0.74790458 0.71209331]
Output_att24_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.73       0.71803279 0.72396694 0.72039474 0.72757475]
Output_att24_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75342466 0.72131148 0.73701843 0.72751323 0.74677529]
Output_att24_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71019108 0.73114754 0.72051696 0.72685789 0.71428571]
Output_att24_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75601375 0.72131148 0.73825503 0.72799471 0.74880871]
Output_att24_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.72       0.70819672 0.71404959 0.71052632 0.71760797]
Output_att24_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76642336 0.68852459 0.7253886  0.70281124 0.74946467]
Output_att24_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.7184466  0.72786885 0.72312704 0.72596468 0.72031149]
Output_att24_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75172414 0.7147541  0.73277311 0.7218543  0.7440273 ]
Output_att24_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69846154 0.7442623  0.72063492 0.73462783 0.70716511]
Output_att24_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75539568 0.68852459 0.72041166 0.70093458 0.74100212]
Output_att24_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74204947 0.68852459 0.71428571 0.69860279 0.73068894]
Output_att24_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72516556 0.71803279 0.72158155 0.71944809 0.72372769]
Output_att24_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.71746032 0.74098361 0.72903226 0.73615635 0.72204473]
Output_att24_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74013158 0.73770492 0.73891626 0.73818898 0.73964497]
Output_att24_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72875817 0.73114754 0.7299509  0.73066841 0.72923479]
Output_att24_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.77067669 0.67213115 0.71803853 0.6897712  0.74872169]
Output_att24_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.71428571 0.72131148 0.7177814  0.71989529 0.7156799 ]
Output_att24_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.69604863 0.75081967 0.72239748 0.73918657 0.7063541 ]
Output_att24_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72491909 0.73442623 0.72964169 0.73250491 0.72680078]
Output_att24_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.71293375 0.74098361 0.7266881  0.73519844 0.71837254]
Output_att24_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70440252 0.73442623 0.71910112 0.72821847 0.71020926]
Output_att24_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76383764 0.67868852 0.71875    0.69416499 0.74514039]
Output_att24_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75912409 0.68196721 0.71848014 0.6961178  0.74232691]
Output_att26_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76124567 0.72131148 0.74074074 0.72895958 0.75290897]
Output_att26_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.6996997  0.76393443 0.73040752 0.75016098 0.71166768]
Output_att26_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74       0.72786885 0.7338843  0.73026316 0.73754153]
Output_att26_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69135802 0.73442623 0.71224165 0.7253886  0.69956277]
Output_att26_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75       0.69836066 0.72325976 0.7081117  0.73907009]
Output_att26_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77186312 0.66557377 0.71478873 0.68442347 0.74797347]
Output_att26_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.70090634 0.76065574 0.72955975 0.74790458 0.71209331]
Output_att26_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.73       0.71803279 0.72396694 0.72039474 0.72757475]
Output_att26_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75342466 0.72131148 0.73701843 0.72751323 0.74677529]
Output_att26_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71019108 0.73114754 0.72051696 0.72685789 0.71428571]
Output_att26_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75601375 0.72131148 0.73825503 0.72799471 0.74880871]
Output_att26_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.72       0.70819672 0.71404959 0.71052632 0.71760797]
Output_att26_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76642336 0.68852459 0.7253886  0.70281124 0.74946467]
Output_att26_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.7184466  0.72786885 0.72312704 0.72596468 0.72031149]
Output_att26_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75172414 0.7147541  0.73277311 0.7218543  0.7440273 ]
Output_att26_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69846154 0.7442623  0.72063492 0.73462783 0.70716511]
Output_att26_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75539568 0.68852459 0.72041166 0.70093458 0.74100212]
Output_att26_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74204947 0.68852459 0.71428571 0.69860279 0.73068894]
Output_att26_5_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.72483221 0.70819672 0.71641791 0.71146245 0.72144289]
Output_att26_5_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.74671053 0.7442623  0.7454844  0.74475066 0.74621959]
Output_att26_5_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77003484 0.72459016 0.74662162 0.73324486 0.76049553]
Output_att26_5_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.75778547 0.71803279 0.73737374 0.72564612 0.74948665]
Output_att26_5_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.74257426 0.73770492 0.74013158 0.73867367 0.74159525]
Output_att26_5_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.71019108 0.73114754 0.72051696 0.72685789 0.71428571]
Output_att26_5_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.76450512 0.73442623 0.74916388 0.74025116 0.75829384]
Output_att26_5_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.75945017 0.72459016 0.74161074 0.73130377 0.75221239]
Output_att26_5_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.72277228 0.71803279 0.72039474 0.71897571 0.72181938]
Output_att26_5_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.77003484 0.72459016 0.74662162 0.73324486 0.76049553]
Output_att26_5_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.7266881  0.74098361 0.73376623 0.73807969 0.72950291]
Output_att26_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72516556 0.71803279 0.72158155 0.71944809 0.72372769]
Output_att26_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.74394464 0.70491803 0.72390572 0.71239231 0.7357974 ]
Output_att26_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74013158 0.73770492 0.73891626 0.73818898 0.73964497]
Output_att26_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72875817 0.73114754 0.7299509  0.73066841 0.72923479]
Output_att26_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.77067669 0.67213115 0.71803853 0.6897712  0.74872169]
Output_att26_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.71428571 0.72131148 0.7177814  0.71989529 0.7156799 ]
Output_att26_8_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.78358209 0.68852459 0.73298429 0.70564516 0.76252723]
Output_att26_8_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.74666667 0.73442623 0.74049587 0.73684211 0.74418605]
Output_att26_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.69604863 0.75081967 0.72239748 0.73918657 0.7063541 ]
Output_att26_8_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77224199 0.71147541 0.74061433 0.72285143 0.75927222]
Output_att26_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.75886525 0.70163934 0.72913118 0.71238349 0.74668528]
Output_att26_8_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.77536232 0.70163934 0.73666093 0.71524064 0.75940383]
Output_att26_8_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.74324324 0.72131148 0.73211314 0.72559367 0.73875084]
Output_att26_8_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.73089701 0.72131148 0.72607261 0.72320842 0.72895958]
Output_att26_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.71293375 0.74098361 0.7266881  0.73519844 0.71837254]
Output_att26_8_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.77210884 0.7442623  0.75792988 0.74966975 0.76637407]
Output_att26_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70440252 0.73442623 0.71910112 0.72821847 0.71020926]
Output_att26_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76383764 0.67868852 0.71875    0.69416499 0.74514039]
Output_att26_8_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.78832117 0.70819672 0.74611399 0.72289157 0.77087794]
Output_att26_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75912409 0.68196721 0.71848014 0.6961178  0.74232691]
Output_att26_8_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.72435897 0.74098361 0.73257699 0.73759791 0.72762395]
Output_att26_8_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.78277154 0.6852459  0.73076923 0.70275723 0.76110706]
Output_att26_8_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.72523962 0.7442623  0.73462783 0.74037834 0.72896596]
Output_att3_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74172185 0.73442623 0.73805601 0.73587385 0.74025116]
Output_att3_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.73139159 0.74098361 0.73615635 0.73904513 0.73329007]
Output_att3_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.73927393 0.73442623 0.73684211 0.73539068 0.73829927]
Output_att3_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.65606936 0.7442623  0.69738863 0.7247765  0.67199526]
Output_att3_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75547445 0.67868852 0.71502591 0.69277108 0.73875803]
Output_att3_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.72108844 0.69508197 0.70784641 0.7001321  0.71573261]
Output_att3_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.71935484 0.73114754 0.72520325 0.72875817 0.72168285]
Output_att3_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72368421 0.72131148 0.72249589 0.72178478 0.72320842]
Output_att3_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75510204 0.72786885 0.74123539 0.7331572  0.74949359]
Output_att3_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71884984 0.73770492 0.72815534 0.73385519 0.72254335]
Output_att3_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74587459 0.74098361 0.74342105 0.74195666 0.74489123]
Output_att3_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74637681 0.67540984 0.7091222  0.68850267 0.7310149 ]
Output_att3_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.73289902 0.73770492 0.73529412 0.7367387  0.73385519]
Output_att3_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72347267 0.73770492 0.73051948 0.73481385 0.72627502]
Output_att3_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.7458194  0.73114754 0.7384106  0.73403555 0.74283811]
Output_att3_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.69817073 0.75081967 0.7235387  0.73966408 0.70810142]
Output_att3_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.71118012 0.75081967 0.73046252 0.74254215 0.71876962]
Output_att3_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75925926 0.67213115 0.71304348 0.68791946 0.7400722 ]
Output_att3_5_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.74324324 0.72131148 0.73211314 0.72559367 0.73875084]
Output_att3_5_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.75085324 0.72131148 0.73578595 0.72703239 0.74475288]
Output_att3_5_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.77617329 0.70491803 0.73883162 0.71810287 0.76079264]
Output_att3_5_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.73986486 0.71803279 0.72878536 0.72229551 0.73539288]
Output_att3_5_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.73509934 0.72786885 0.73146623 0.72930355 0.73364177]
Output_att3_5_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.73666667 0.72459016 0.73057851 0.72697368 0.73421927]
Output_att3_5_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.75415282 0.7442623  0.74917492 0.74621959 0.75215374]
Output_att3_5_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.79924242 0.69180328 0.74165202 0.71091644 0.77516532]
Output_att3_5_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.69552239 0.76393443 0.728125   0.74919614 0.70820669]
Output_att3_5_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.7761194  0.68196721 0.72600349 0.69892473 0.75526507]
Output_att3_5_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.73941368 0.7442623  0.74183007 0.74328749 0.74037834]
Output_att3_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72459016 0.72459016 0.72459016 0.72459016 0.72459016]
Output_att3_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72100313 0.75409836 0.73717949 0.74723847 0.72738773]
Output_att3_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.73225806 0.7442623  0.73821138 0.74183007 0.73462783]
Output_att3_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.76208178 0.67213115 0.71428571 0.68838146 0.74221579]
Output_att3_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.75       0.70819672 0.72849916 0.71618037 0.74124914]
Output_att3_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.74825175 0.70163934 0.72419628 0.71049137 0.7384403 ]
Output_att3_8_0.6075_0.45_0.5_fn_prop_corrected.txt:Final Results:  [0.73489933 0.71803279 0.72636816 0.72134387 0.73146293]
Output_att3_8_0.6075_0.5_0.5_fn_prop_corrected.txt:Final Results:  [0.71875    0.75409836 0.736      0.74675325 0.72555205]
Output_att3_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.73927393 0.73442623 0.73684211 0.73539068 0.73829927]
Output_att3_8_0.6075_0.62_0.5_fn_prop_corrected.txt:Final Results:  [0.78494624 0.71803279 0.75       0.73048699 0.7705841 ]
Output_att3_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.71656051 0.73770492 0.726979   0.73337679 0.72069186]
Output_att3_8_0.6075_0.65_0.5_fn_prop_corrected.txt:Final Results:  [0.77256318 0.70163934 0.73539519 0.71476286 0.75725407]
Output_att3_8_0.6075_0.6_0.5_fn_prop_corrected.txt:Final Results:  [0.73489933 0.71803279 0.72636816 0.72134387 0.73146293]
Output_att3_8_0.6075_0.7_0.5_fn_prop_corrected.txt:Final Results:  [0.75337838 0.73114754 0.74209651 0.73548813 0.74882471]
Output_att3_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74666667 0.73442623 0.74049587 0.73684211 0.74418605]
Output_att3_8_0.6075_0.85_0.5_fn_prop_corrected.txt:Final Results:  [0.75874126 0.71147541 0.73434856 0.72045153 0.74879227]
Output_att3_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.72164948 0.68852459 0.70469799 0.69490404 0.71477195]
Output_att3_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76811594 0.69508197 0.72977625 0.70855615 0.7523066 ]
Output_att3_8_0.6075_0.87_0.5_fn_prop_corrected.txt:Final Results:  [0.72929936 0.75081967 0.73990307 0.7464146  0.73350416]
Output_att3_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.7751938  0.6557377  0.71047957 0.67658999 0.74794316]
Output_att3_8_0.6075_0.8_0.5_fn_prop_corrected.txt:Final Results:  [0.78388278 0.70163934 0.74048443 0.71667783 0.76592699]
Output_att3_8_0.6075_0.9_0.5_fn_prop_corrected.txt:Final Results:  [0.77777778 0.71147541 0.74315068 0.72381588 0.7635468 ]
Output_att3_8_0.6075_1_0.5_fn_prop_corrected.txt:Final Results:  [0.75874126 0.71147541 0.73434856 0.72045153 0.74879227]
Output_att5_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.70121951 0.75409836 0.72669826 0.74289406 0.71119357]
Output_att5_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.69461078 0.76065574 0.72613459 0.74646075 0.70688605]
Output_att5_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.7585034  0.73114754 0.74457429 0.73645971 0.75286968]
Output_att5_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.75789474 0.70819672 0.73220339 0.71760797 0.74740484]
Output_att5_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74489796 0.71803279 0.7312187  0.72324967 0.73936529]
Output_att5_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77777778 0.66557377 0.71731449 0.68534774 0.75240919]
Output_att5_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74086379 0.73114754 0.7359736  0.73307035 0.73889993]
Output_att5_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.73630137 0.70491803 0.72026801 0.71097884 0.72980312]
Output_att5_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.72903226 0.74098361 0.73495935 0.73856209 0.73139159]
Output_att5_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.71428571 0.73770492 0.72580645 0.73289902 0.71884984]
Output_att5_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74733096 0.68852459 0.71672355 0.69953364 0.73477957]
Output_att5_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.76492537 0.67213115 0.71553229 0.68884409 0.74437182]
Output_att5_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74827586 0.71147541 0.72941176 0.71854305 0.74061433]
Output_att5_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72347267 0.73770492 0.73051948 0.73481385 0.72627502]
Output_att5_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.7125     0.74754098 0.7296     0.74025974 0.7192429 ]
Output_att5_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.75086505 0.71147541 0.73063973 0.71901922 0.74264203]
Output_att5_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74       0.72786885 0.7338843  0.73026316 0.73754153]
Output_att5_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77220077 0.6557377  0.70921986 0.67613252 0.74571216]
Output_att5_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.76811594 0.69508197 0.72977625 0.70855615 0.7523066 ]
Output_att5_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.75503356 0.73770492 0.74626866 0.74110672 0.75150301]
Output_att5_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.72292994 0.7442623  0.73344103 0.7398957  0.72709801]
Output_att5_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70793651 0.73114754 0.71935484 0.72638436 0.71246006]
Output_att5_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.72131148 0.72131148 0.72131148 0.72131148 0.72131148]
Output_att5_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.73144876 0.67868852 0.70408163 0.68862275 0.72025052]
Output_att5_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.79400749 0.69508197 0.74125874 0.71284465 0.77203205]
Output_att5_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72847682 0.72131148 0.72487644 0.72273325 0.72703239]
Output_att5_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74747475 0.72786885 0.73754153 0.73170732 0.74346952]
Output_att5_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.68452381 0.75409836 0.71762871 0.73907455 0.69739236]
Output_att5_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76173285 0.69180328 0.72508591 0.70474282 0.74663836]
Output_att5_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.78039216 0.65245902 0.71071429 0.67457627 0.7509434 ]
Output_att7_1_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.75172414 0.7147541  0.73277311 0.7218543  0.7440273 ]
Output_att7_1_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.72727273 0.73442623 0.73083197 0.73298429 0.72869226]
Output_att7_1_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.72204473 0.74098361 0.73139159 0.73711676 0.72575466]
Output_att7_1_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70662461 0.73442623 0.72025723 0.72869226 0.71201526]
Output_att7_1_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.73913043 0.72459016 0.73178808 0.72745227 0.73617588]
Output_att7_1_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75276753 0.66885246 0.70833333 0.68410463 0.73434125]
Output_att7_3_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.71935484 0.73114754 0.72520325 0.72875817 0.72168285]
Output_att7_3_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.7120743  0.75409836 0.73248408 0.74530136 0.72010019]
Output_att7_3_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.73913043 0.72459016 0.73178808 0.72745227 0.73617588]
Output_att7_3_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.70253165 0.72786885 0.71497585 0.72265625 0.70745698]
Output_att7_3_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76173285 0.69180328 0.72508591 0.70474282 0.74663836]
Output_att7_3_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.76865672 0.67540984 0.71902269 0.6922043  0.7480029 ]
Output_att7_4_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.71786834 0.75081967 0.73397436 0.7439896  0.72422517]
Output_att7_4_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.69811321 0.72786885 0.71268058 0.72171651 0.7038681 ]
Output_att7_4_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.75       0.72786885 0.73876872 0.73218997 0.74546676]
Output_att7_4_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.76449275 0.69180328 0.72633391 0.7052139  0.74875798]
Output_att7_4_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.76241135 0.70491803 0.73253833 0.71571238 0.75017446]
Output_att7_4_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.75645756 0.67213115 0.71180556 0.68745808 0.73794096]
Output_att7_6_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.74662162 0.72459016 0.73544093 0.72889182 0.7421088 ]
Output_att7_6_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.71339564 0.75081967 0.73162939 0.74302401 0.72057898]
Output_att7_6_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.74832215 0.73114754 0.73963516 0.7345191  0.74482298]
Output_att7_6_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.7218543  0.7147541  0.71828666 0.71616294 0.720423  ]
Output_att7_6_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.72491909 0.73442623 0.72964169 0.73250491 0.72680078]
Output_att7_6_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.77559055 0.64590164 0.70483005 0.66824966 0.74564724]
Output_att7_8_0.6075_0.62_0.4_fn_prop_corrected.txt:Final Results:  [0.72903226 0.74098361 0.73495935 0.73856209 0.73139159]
Output_att7_8_0.6075_0.62_0.75_fn_prop_corrected.txt:Final Results:  [0.70754717 0.73770492 0.7223114  0.73146944 0.71337984]
Output_att7_8_0.6075_0.85_0.4_fn_prop_corrected.txt:Final Results:  [0.73442623 0.73442623 0.73442623 0.73442623 0.73442623]
Output_att7_8_0.6075_0.85_0.75_fn_prop_corrected.txt:Final Results:  [0.68902439 0.74098361 0.71406003 0.72997416 0.69882498]
Output_att7_8_0.6075_0.87_0.4_fn_prop_corrected.txt:Final Results:  [0.74237288 0.71803279 0.73       0.72277228 0.73737374]
Output_att7_8_0.6075_0.87_0.75_fn_prop_corrected.txt:Final Results:  [0.76515152 0.66229508 0.71001757 0.68059299 0.7421014 ]

"""

sorted([float(l.split(":")[2].strip().split()[2]) if l.startswith("Output") else 0 for l in output.split("\n")], reverse=True)

[0.75792988,
 0.75792988,
 0.75,
 0.74917492,
 0.74916388,
 0.74916388,
 0.74914089,
 0.74873524,
 0.74706868,
 0.74706868,
 0.74662162,
 0.74662162,
 0.74662162,
 0.74662162,
 0.74662162,
 0.74657534,
 0.74626866,
 0.74611399,
 0.74611399,
 0.7454844,
 0.7454844,
 0.74457429,
 0.74450085,
 0.74371859,
 0.74342105,
 0.74315068,
 0.74295191,
 0.74257426,
 0.74257426,
 0.74209651,
 0.74209651,
 0.74209651,
 0.74188034,
 0.74183007,
 0.74165202,
 0.74161074,
 0.74161074,
 0.74125874,
 0.74123539,
 0.74123539,
 0.74111675,
 0.74086379,
 0.74074074,
 0.74074074,
 0.74074074,
 0.74061433,
 0.74061433,
 0.74049587,
 0.74049587,
 0.74049587,
 0.74048443,
 0.74038462,
 0.74025974,
 0.74025974,
 0.74013158,
 0.74013158,
 0.74013158,
 0.74,
 0.73990307,
 0.73977087,
 0.73963516,
 0.73891626,
 0.73891626,
 0.73891626,
 0.73891626,
 0.73883162,
 0.73876872,
 0.7384106,
 0.73825503,
 0.73825503,
 0.73825503,
 0.73821138,
 0.73805601,
 0.73786408,
 0.73786408,
 0.73754153,
 0.73737374,
 0.73737374,
 

In [ ]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [ ]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

In [7]:
[1/3 for i in range(3)]

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [64]:
max_paths

1

In [ ]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




In [ ]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

In [ ]:
inp_spellchecked[730], inp[731]

In [ ]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

In [ ]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

In [ ]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

In [ ]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

In [ ]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [ ]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

In [ ]:
fn

In [ ]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [ ]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

In [ ]:
Ontology("conference_ontologies/conference.owl").triples

In [ ]:
cos_sim2